In [1]:
!pip install pillow

In [2]:
import time
import os
import multiprocessing as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# Layer name to inspect
layer_name = 'block5_conv2'
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
epochs = 30
step_size = .05
# filter_index = 1

# Create a connection between the input and the target layer
model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')


# Initiate random noise
input_img_data = np.random.random((1, 224, 224, 3))
input_img_data = (input_img_data - 0.5) * 20 + 128.

# Cast random noise from np.float64 to tf.float32 Variable




In [3]:
targets = []
for i, layer in enumerate(model.layers):
    if 'conv' in layer.name:
        targets.append(layer.name)
targets

['block1_conv1',
 'block1_conv2',
 'block2_conv1',
 'block2_conv2',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3']

In [5]:
#%%time
for layer_name in targets[0:1]:
    print(f'generating dataset {layer_name}')
    submodel = tf.keras.models.Model([model.inputs[0]], 
                                     [model.get_layer(layer_name).output])
        
    batch = submodel.output.get_shape()[-1]
    input_img_data = np.random.random((batch, 224, 224, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128.
    input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))
    #input_img_list = tf.split(input_img_data, batch, axis=0)
    #zero_init = tf.zeros_initializer()
    
    #print(loss_value)
    for _ in range(epochs):
        #print(tf.split(input_img_data, batch, axis=0))
        grads = []
        with tf.GradientTape() as tape:
            tape.watch(input_img_data)
            loss_value = []
            outputs = tf.dtypes.cast(submodel(preprocess_input(input_img_data)), tf.float32)
            
            # calculate loss with respect to each channel
            #for i in range(batch):                 
            loss = tf.reduce_mean(outputs, axis=(1,2,3))
            loss = tf.reduce_mean(loss)
            print(loss)   
            
        # get gradients of image batch with respect to each lost
        grad = tape.gradient(loss, input_img_data)
        print(grad[0][0][0][0])
#         for i in range(batch): 
#             grad = tape.gradient(loss_value[i], input_img_data)
        normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
        input_img_data.assign_add(grad * step_size)
            #tf.compat.v1.scatter_add(input_img_data, i, normalized_grad * step_size)
        del tape
        

        #normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
        #input_img_data.assign_add(normalized_grads * step_size)
        


generating dataset block1_conv1
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(7.0395236, s

In [5]:
grads[1][0,:,:,0]

IndexError: list index out of range

In [ ]:
plt.imshow(input_img_data.numpy()[2,:,:,:].astype(np.uint8))

In [ ]:
Image.fromarray(input_img_data.numpy()[0,:,:,:].astype(np.uint8)).save(fp='test.jpg')

In [ ]:
1.49